In [1]:
import numpy as np
import pandas as pd
import functools
from typing import Optional, Callable
np.random.seed(1)

## Задание 5 

### Вариант 2

## КФ Гаусса

In [2]:
N = np.array([4, 6, 7, 8])

In [3]:
left, right = 1, 10

### $f(x) = \frac{1}{(1 + x^2)(4 + 3x^2)}$

In [4]:
def func1(
    arg: float
    ) -> float:
    return 1 / ((1+arg*arg)*(4+3*arg*arg))

### Интеграл от заданной функции

### $\int f(x)dx = ctg(x) - \frac{\sqrt3}{2}ctg(\frac{\sqrt3x}{2})$

In [5]:
I1 = ((np.arctan(right)) - np.sqrt(3)/2 * np.arctan(np.sqrt(3)/2 * right))

In [6]:
I2 = ((np.arctan(left)) - np.sqrt(3)/2 * (np.arctan(np.sqrt(3)/2 * left)))

In [7]:
I2

0.16729471993100997

In [8]:
I = I1 - I2
I

0.04304250879311655

###  Полином Лежандра

In [9]:
@functools.lru_cache(maxsize=None)
def legendre_polynom(
    arg: float,
    n: int
    ) -> float:
    
    if n == 0:
        return 1
    if n == 1:
        return arg
    
    return (2*n-1)/n * legendre_polynom(arg, n-1)*arg - (n-1)/n * legendre_polynom(arg, n-2)

In [10]:
def firstderiv_legendre_polynom(
    arg: float,
    n: int
    ) -> float:
    
    if n == 0:
        return 0
    if n == 1:
        return 1
    
    return n / (1 - arg*arg) * (legendre_polynom(arg, n - 1) - arg * legendre_polynom(arg, n))

In [11]:
def secondderiv_legendre_polynom(
    arg: float,
    n: int
    ) -> float:
    
    if n == 0 and n == 1:
        return 0
    
    return 2*n*arg/((1 -arg*arg)*(1-arg*arg)) \
         *(legendre_polynom(arg, n - 1) \
         -arg * legendre_polynom(arg, n)) \
         +n / (1 - arg*arg) * (firstderiv_legendre_polynom(arg, n-1) \
         -legendre_polynom(arg, n) - arg*firstderiv_legendre_polynom(arg, n))
        
        
                

### Коэффициенты Гаусса

In [12]:
def gauss_coefficient(
    nodes: np.ndarray,
    legendre_polynom: Optional[Callable],
    n: int
    ) -> np.ndarray:
    
    A = []
    
    for node in nodes:
        P = legendre_polynom(node, n-1)
        coef = 2*(1-node*node) / (n*n*P*P)
        A.append((node, coef))
    return np.array(A)

### Найдем корни многочлена Лежандра

In [13]:
def separation_method(
    getFunc: Optional[Callable],
    N_nodes: int,
    left: float,
    right: float,
    N=1000
    ) -> np.ndarray:

    ans = []
    
    t = (right - left) / N
    x1 = left
    x2 = left + t 
    y1 = 0
    y2 = 0
    counter = 0
    while (x2 <= right):
        y1 = getFunc(x1, N_nodes)
        y2 = getFunc(x2, N_nodes)
        if (y1 * y2 <= 0):
            pair = (x1, x2)
            ans.append(pair)
            counter+=1
        x1 = x2
        x2 += t
    
    return ans

In [14]:
def secant_method(
    getFunc: Optional[Callable],
    N: int,
    lBound: float,
    rBound: float,
    epsilon: float
    ):
    if (getFunc(lBound, N) < epsilon):
        return lBound
    if (getFunc(rBound, N) < epsilon):
        return rBound
    
    prevSol = lBound + np.random.rand() * (rBound - lBound)
    while (secondderiv_legendre_polynom(prevSol, N) * getFunc(prevSol, N) <= 0):
         prevSol = lBound + np.random.rand() * (rBound - lBound)
    
    currSol = lBound + np.random.rand() * (rBound - lBound)
    nextSol = currSol - (getFunc(currSol, N) * (currSol - prevSol)
                         / (getFunc(currSol, N) - getFunc(prevSol, N)))
    
    step = 0
    while(np.abs(currSol - prevSol) >= epsilon):
        prevSol = currSol
        currSol = nextSol
        nextSol = currSol - (getFunc(currSol, N) * (currSol - prevSol)
                         / (getFunc(currSol, N) - getFunc(prevSol, N)))
        step += 1
    return currSol

In [15]:
def get_coefficient(
    N: int,
    left: float,
    right: float,
    epsilon=1e-12
    ) -> np.ndarray:
    segments = separation_method(legendre_polynom, N, left, right)
    nodes = []
    for l, r in segments:
        nodes.append(secant_method(legendre_polynom, N, l, r, epsilon))
    nodes = np.array(nodes)
    return gauss_coefficient(nodes, legendre_polynom, N)

## Узлы <-> Коэффициенты при $N=\overline{1,8}$

In [16]:
data = {}
for n in range(1, 9):
    
    nodes_cf = get_coefficient(n, -1, 1)
    df = pd.DataFrame(np.array([nodes_cf[:,0], nodes_cf[:,1]]),
                     columns=np.arange(1, n + 1),
                     index=['Node', 'CF'])
    data[n] = df

## N=1

In [17]:
data[1]

,1
Node,-0.002000
CF,1.999992


## N=2

In [18]:
data[2]

,1,2
Node,-0.576000,0.576000
CF,1.007041,1.007041


## N=3

In [19]:
data[3]

,1,2,3
Node,-0.776000,8.743006e-16,0.774000
CF,0.543625,8.888889e-01,0.560718


## N=4

In [20]:
data[4]

,1,2,3,4
Node,-0.860000,-0.340000,0.340000,0.860000
CF,0.361329,0.652098,0.652098,0.361329


## N=5

In [21]:
data[5]

,1,2,3,4,5
Node,-0.908000,-0.538000,-0.002000,0.540000,0.906000
CF,0.212404,0.480681,0.568932,0.472046,0.239535


## N=6

In [22]:
data[6]

,1,2,3,4,5,6
Node,-0.932000,-0.662000,-0.238000,0.238000,0.662000,0.932000
CF,0.179617,0.356119,0.468947,0.468947,0.356119,0.179617


## N=7

In [23]:
data[7]

,1,2,3,4,5,6,7
Node,-0.950000,-0.74000,-0.406000,8.743006e-16,0.404000,0.742000,0.9480
CF,0.113253,0.29135,0.381371,4.179592e-01,0.387422,0.276281,0.1541


## N=8

In [24]:
data[8]

,1,2,3,4,5,6,7,8
Node,-0.960000,-0.798000,-0.524000,-0.184000,0.184000,0.524000,0.798000,0.960000
CF,0.108029,0.211177,0.320128,0.361991,0.361991,0.320128,0.211177,0.108029


In [25]:
comp_int = []

In [26]:
def compute_integral(
    nodes_cf,
    left,
    right,
    func
    ) -> float:
    integral = 0
    for node, cf in nodes_cf:
        integral += cf * func((left + right)/2 + (right - left)/2 * node)
    integral *= (right - left)/2
    return integral

## N = 4

In [27]:
nodes_cf = get_coefficient(N[0], -1, 1)

In [28]:
value1 = compute_integral(nodes_cf, left, right, func1)
comp_int.append(value1)
value1

0.04100818127840956

## N = 6

In [29]:
nodes_cf = get_coefficient(N[1], -1, 1)
comp_int.append(compute_integral(nodes_cf, left, right, func1))

In [30]:
comp_int[1]

0.04432915022211627

## N = 7

In [31]:
nodes_cf = get_coefficient(N[2], -1, 1)
comp_int.append(compute_integral(nodes_cf, left, right, func1))

In [32]:
comp_int[2]

0.04022415385192088

## N = 8

In [33]:
nodes_cf = get_coefficient(N[3], -1, 1)
comp_int.append(compute_integral(nodes_cf, left, right, func1))

In [34]:
comp_int[3]

0.04402355001884226

In [35]:
comp_int = np.array(comp_int)

In [36]:
comp_int

array([0.04100818, 0.04432915, 0.04022415, 0.04402355])

In [37]:
error = np.array(comp_int - I)
df = pd.DataFrame(np.array([comp_int, error]),
                  columns=N,
                  index=np.array(['Value', 'Error']))
df

,4,6,7,8
Value,0.041008,0.044329,0.040224,0.044024
Error,-0.002034,0.001287,-0.002818,0.000981


## КФ Мелера

### $f(x) = e^{2x}$

In [38]:
def func2(
    arg: float
    ) -> float:
    
    return np.exp(2*arg)

In [39]:
def p(
    arg: float
    ) -> float:
    
    return 1 / np.sqrt(1-arg*arg)

## Интеграл

In [40]:
I = 7.161_528_439_050_256_662

In [59]:
N = list(map(int, input().split()))

4 6 8


## Многочлен Чебышева первого рода

In [60]:
@functools.lru_cache(maxsize=None)
def chebishev_polynom(
    arg: float,
    n: int
    ) -> float:
    
    if n == 0:
        return 1
    if n == 1:
        return arg
    return 2 * arg * chebishev_polynom(arg, n - 1) - chebishev_polynom(arg, n - 2) 

In [61]:
n1_cf = np.pi / N[0]
n1_nodes = [np.cos(np.pi*(i - 1/2)/N[0]) for i in range(1, N[0] + 1)]

In [62]:
print("N1")
print(f"coef={n1_cf}")
print(f"nodes={n1_nodes}")

N1
coef=0.7853981633974483
nodes=[0.9238795325112867, 0.38268343236508984, -0.3826834323650897, -0.9238795325112867]


In [63]:
n2_cf = np.pi / N[1]
n2_nodes = [np.cos(np.pi*(i - 1/2)/N[1]) for i in range(1, N[1] + 1)]

In [64]:
print("N2")
print(f"coef={n2_cf}")
print(f"nodes={n2_nodes}")

N2
coef=0.5235987755982988
nodes=[0.9659258262890683, 0.7071067811865476, 0.25881904510252074, -0.25881904510252063, -0.7071067811865475, -0.9659258262890682]


In [65]:
n3_cf = np.pi / N[2]
n3_nodes = [np.cos(np.pi*(i - 1/2)/N[2]) for i in range(1, N[2] + 1)]

In [66]:
print("N3")
print(f"coef={n3_cf}")
print(f"nodes={n3_nodes}")

N3
coef=0.39269908169872414
nodes=[0.9807852804032304, 0.8314696123025452, 0.5555702330196023, 0.19509032201612833, -0.1950903220161282, -0.555570233019602, -0.8314696123025453, -0.9807852804032304]


In [67]:
integral1 = 0
for i in n1_nodes:
    integral1 += func2(i)
integral1 *= n1_cf
error1 = integral1 - I

In [68]:
integral2= 0
for i in n2_nodes:
    integral2 += func2(i)
integral2 *= n2_cf
error2 = integral2 - I

In [69]:
integral3= 0
for i in n3_nodes:
    integral3 += func2(i)
integral3 *= n3_cf
error3 = integral3 - I

In [70]:
df = pd.DataFrame(np.array([np.array([integral1, integral2, integral3]),
                           np.array([error1, error2, error3])]),
                  columns=[f'N1={N[0]}', f'N2={N[1]}', f'N3={N[2]}'],
                  index=['Value', 'Error'])
                  

In [71]:
df

,N1=4,N2=6,N3=8
Value,7.161354,7.161528e+00,7.161528e+00
Error,-0.000174,-1.416312e-08,-3.179679e-13
